In [57]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from IPython.display import display

In [59]:
driver = webdriver.Chrome()

In [60]:
api_key = "RGAPI-ec132e4d-77ce-4de1-bb45-fdaf011ba5b6"

In [61]:
leagueacc_url="https://porofessor.gg/live/na/Cytosine-NA1"

In [62]:
driver.get(leagueacc_url)
ally_url=[]
ally_kda_list=[]
enemy_kda_list=[]

In [66]:
ally_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader blue')]"))

In [67]:
enemy_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader red')]"))

In [ ]:
playerTag="SquatsForThots#NA1"
playerName, playerTag = playerTag.split("#")
get_puuid_url=f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{playerName}/{playerTag}" + '?api_key=' + api_key
get_puuid_raw=requests.get(get_puuid_url)
while get_puuid_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        get_puuid_raw = requests.get(get_puuid_url)
get_puuid = get_puuid_raw.json()['puuid']
get_current_match_url = f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{get_puuid}" + '?api_key=' + api_key
get_current_match_raw = requests.get(get_current_match_url)
while get_current_match_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        get_current_match_raw = requests.get(get_current_match_url)
get_current_match_puuids = get_current_match_raw.json()['participants']

for playerNumbers in range(5):
    print(get_current_match_puuids[playerNumbers].get('riotId'))
    player_puuid=get_current_match_puuids[playerNumbers].get('puuid')
    matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_puuid}/ids?start=0&count=30" + '&api_key=' + api_key
    matches_info_raw = requests.get(matches_api_url)
    while matches_info_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        matches_info_raw = requests.get(matches_api_url)
    matches_info= list(matches_info_raw.json())
        
    for match in matches_info:
        match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
        match_info_raw = requests.get(match_api_url)
        while match_info_raw.status_code == 429:
            print("Sleeping")
            time.sleep(10)
            match_info_raw = requests.get(match_api_url)
        match_info= match_info_raw.json()
        participantIndex = match_info['metadata']['participants'].index(player_puuid)
        matchData=match_info['info']['participants'][participantIndex]
        if (match_info['info']['queueId']==420) and (matchData['timePlayed']>180):
            solo_testing_key_stats = ['championName','teamPosition','kills', 'deaths', 'assists', 'damageDealtToBuildings', 'deaths', 'visionScore','doubleKills', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'goldEarned', 'inhibitorKills', 'inhibitorTakedowns', 'killingSprees', 'objectivesStolen', 'totalMinionsKilled', 'totalDamageDealt', 'turretKills','win']
            team_testing_key_stats = ['kda', 'damagePerMinute', 'goldPerMinute', 'teamDamagePercentage','dodgeSkillShotsSmallWindow', 'takedowns']
            support_solo_key_stats = ['detectorWardsPlaced']
            support_team_key_stats = ['controlWardsPlaced', 'completeSupportQuestInTime']
            jungle_solo_key_stats = ['neutralMinionsKilled', 'detectorWardsPlaced', 'dragonKills']
            jungle_team_key_stats = ['jungleCsBefore10Minutes']
            base_stats = {
                    **{stat: matchData[stat] for stat in solo_testing_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in team_testing_key_stats}
                }
            if matchData['teamPosition']== 'UTILITY':
                role_stats={
                    **{stat: matchData[stat] for stat in support_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in support_team_key_stats}
                }
            elif matchData['teamPosition']== 'JUNGLE':
               role_stats={
                    **{stat: matchData[stat] for stat in jungle_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in jungle_team_key_stats}
                }
            else:
                role_stats={}
            timePlayed = f"{matchData['timePlayed'] // 60}:{matchData['timePlayed'] % 60:02d}"
            df=pd.DataFrame([{**base_stats, **role_stats}])
            df.insert(0, 'timePlayed', timePlayed)
            df.index = ['']
            pd.set_option('display.max_columns', None)  
            pd.set_option('display.expand_frame_repr', False)
            # pd.set_option('display.width', 1000)
            display(df)
            # print(match_info['info']['participants'][participantIndex], end =" ")
            # print(match_info['info']['participants'][participantIndex]['kills'], end =" ")
            # print(match_info['info']['participants'][participantIndex]['deaths'], end =" ")
            # print(match_info['info']['participants'][participantIndex]['assists'])

SquatsForThots#NA1


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,neutralMinionsKilled,detectorWardsPlaced,dragonKills,jungleCsBefore10Minutes
,32:58,MasterYi,JUNGLE,8,9,6,72,21,2,True,False,False,False,12618,1,1,3,0,13,247284,0,True,1.555556,537.210923,382.544939,0.167317,0,14,159,0,3,69.25


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,neutralMinionsKilled,detectorWardsPlaced,dragonKills,jungleCsBefore10Minutes
,36:49,MasterYi,JUNGLE,20,11,3,3085,19,3,False,False,False,False,18703,0,0,7,0,60,320140,1,False,2.090909,1587.92453,507.867526,0.299147,2,23,152,0,2,68.0


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,neutralMinionsKilled,detectorWardsPlaced,dragonKills,jungleCsBefore10Minutes
,29:39,MasterYi,JUNGLE,5,9,1,0,20,0,False,True,False,False,10084,0,0,1,0,27,182791,0,False,0.666667,520.808087,339.958049,0.184301,5,6,132,0,1,60.75


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,neutralMinionsKilled,detectorWardsPlaced,dragonKills,jungleCsBefore10Minutes
,22:15,MasterYi,JUNGLE,16,3,6,2034,18,3,False,False,False,False,13372,0,0,4,0,24,200556,0,True,7.333333,1264.023251,600.527255,0.430134,2,22,137,0,3,59.25


...

,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,31:10,Mordekaiser,TOP,5,8,5,383,4,0,False,False,False,False,10396,0,0,2,0,156,122819,1,False,1.25,962.451662,333.414198,0.258256,7,10


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,33:31,Mordekaiser,TOP,2,8,5,3069,4,0,False,False,False,False,11750,0,0,0,0,208,146439,2,False,0.875,569.325356,350.386221,0.171109,0,7


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,35:05,Mordekaiser,TOP,4,7,1,2587,1,0,False,False,False,False,12226,0,0,1,0,210,149956,1,False,0.714286,588.229836,348.327551,0.143984,5,5


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,28:10,Mordekaiser,TOP,1,8,3,1605,1,0,False,False,False,False,7538,0,0,0,0,145,93330,0,False,0.5,638.896565,267.482156,0.176474,7,4


,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns
,31:31,Mordekaiser,TOP,3,7,1,1896,6,0,False,False,False,False,9408,0,0,1,0,178,104138,0,False,0.571429,504.174466,298.436175,0.17691,0,4


In [ ]:
requests.get(api)

In [ ]:
# for player in ally_tag:
#     print(player)
#     name, tag = player.split("#")
#     puuid_api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}" + '?api_key=' + api_key
#     # player_info_raw = requests.get(puuid_api_url)
#     player_info = requests.get(puuid_api_url).json()['puuid']
#     matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_info}/ids?start=0&count=20" + '&api_key=' + api_key
#     # matches_info_raw = requests.get(matches_api_url)
#     matches_info = list(requests.get(matches_api_url).json())
#     for match in matches_info:
#         match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
#         # match_info_raw = requests.get(match_api_url)
#         match_info =requests.get(match_api_url).json()
#         participantIndex = match_info['metadata']['participants'].index(player_info)
#         print(match_info['info']['participants'][participantIndex]['kills'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['deaths'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['assists'])

In [ ]:
for player in ally_url:
    print(player)

In [429]:
# for player in ally_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# for player in enemy_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------
# for player in ally_url:
#     driver.get(player)
#     driver.get(ally_url[0])
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     ally_kda_list.append(kda)
#     print(kda)kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
# deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
# assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
# kda=(kills+assists)/deaths
# ally_kda_list.append(kda)
# print(kda)
# for player in enemy_url:
#     driver.get(player)
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     enemy_kda_list.append(kda)
#     print(kda)
# ally_average_kda=sum(ally_kda_list)/5
# enemy_average_kda=sum(enemy_kda_list)/5